##  Popcount - Python MMIO

Version: 2022.0
 

In [ ]:
import importlib
import numpy as np
import timeit as tt

In [ ]:
class SoftwarePopcount():
    
    def name(self):
        return "Software_Popcount"
    
    def countInt(self, n):        
        w = 0
        while (n):
            w += 1
            n &= n - 1
        return w
         
    def countArray (self, buf):
        total_ones = 0
        for b in buf:
            total_ones += self.countInt(b)
        return total_ones
        
    def countFile(self,file):
        f = open(file, "r")
        buf = np.fromfile(f, dtype=np.uint32)
        return self.countArray(buf)      
    

In [1]:
import numpy as np

from pynq import Overlay
from pynq import MMIO

class HardwarePopcount():
    
    def __init__(self):
        self.overlay = Overlay('bitstream.bit')        
        self.mmio = self.overlay.axi_popcount_0.S_AXI_LITE        

    def name(self):
        return "Hardware_Popcount"
    
    def countInt(self, n): 
        self.mmio.write(0x0, 0x1) # reset hw count
        self.mmio.write(0x4, int(n))
        return self.mmio.read(0x4) # read hw count
    
    def countArray (self, buf):
        self.mmio.write(0x0, 0x1) # reset hw count
        for b in buf:
            self.mmio.write(0x4, int(b))
        return self.mmio.read(0x4) # read hw count
            
    def countFile(self,file):
        f = open(file, "r")
        buf = np.fromfile(f, dtype=np.uint32)
        return self.countArray(buf) 

ModuleNotFoundError: No module named 'pynq'

In [ ]:

for counter in [ SoftwarePopcount(), HardwarePopcount()]:
    print ()
    print ("Testing Counter: " + str(counter.name()))
    print ()
    for i in range(17):
        print ('value:' + str(i) + ' total_ones:' + str(counter.countInt(i)))

    x = list(range(17))
    print ('Array: ' + str(list(x)) + '\n Total Ones: ' + str(counter.countArray(x)))

In [ ]:
sw_counter = SoftwarePopcount()

def count_zeros_sw():
    print ("Found " + str(sw_counter.countFile("zeros.bin")) + " Ones")
def count_ones_sw():
    print ("Found " + str(sw_counter.countFile("ones.bin")) + " Ones")
def count_tiny_sw():
    print ("Found " + str(sw_counter.countFile("tiny.bin")) + " Ones")
def count_small_sw():
    print ("Found " + str(sw_counter.countFile("small.bin")) + " Ones")
def count_medium_sw():
    print ("Found " + str(sw_counter.countFile("medium.bin")) + " Ones")
def count_large_sw():
    print ("Found " + str(sw_counter.countFile("large.bin")) + " Ones")

In [ ]:
# 'ones.bin' contains 256B of all binary 1's
print("Timing 'ones.bin'")
time = tt.timeit(count_ones_sw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'zeros.bin' contains 256B of all binary 0's
print("Timing 'zeros.bin'")
time = tt.timeit(count_zeros_sw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'tiny.bin' contains 256B of random 1's
print("Timing 'tiny.bin'")
time = tt.timeit(count_tiny_sw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'small.bin' contains 4KB of random 1's
print("Timing 'small.bin'")
time = tt.timeit(count_small_sw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'medium.bin' contains 1M of random 1's
print("Timing 'medium.bin'")
time = tt.timeit(count_medium_sw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

### you don't really want to do this 
### unless you have a lot of free time...
### 'large.bin' contains 10M of random 1's
#print("Timing 'large.bin'")
#time = tt.timeit(count_large_sw, number=1)
#print("Total Time:" + str(time) + " seconds")
#print()

In [ ]:
hw_counter = HardwarePopcount()


def count_ones_hw():
    print ("Found " + str(hw_counter.countFile("ones.bin")) + " Ones")  
def count_zeros_hw():
    print ("Found " + str(hw_counter.countFile("zeros.bin")) + " Ones")  
def count_tiny_hw():
    print ("Found " + str(hw_counter.countFile("tiny.bin")) + " Ones")    
def count_small_hw():
    print ("Found " + str(hw_counter.countFile("small.bin")) + " Ones")
def count_medium_hw():
    print ("Found " + str(hw_counter.countFile("medium.bin")) + " Ones")
def count_large_hw():
    print ("Found " + str(hw_counter.countFile("large.bin")) + " Ones")

In [ ]:
# 'ones.bin' contains 256B of all binary 1's
print("Timing 'ones.bin'")
time = tt.timeit(count_ones_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'zeros.bin' contains 256B of all binary 0's
print("Timing 'zeros.bin'")
time = tt.timeit(count_zeros_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'tiny.bin' contains 256B of random 1's
print("Timing 'tiny.bin'")
time = tt.timeit(count_tiny_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'small.bin' contains 4KB of random 1's
print("Timing 'small.bin'")
time = tt.timeit(count_small_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

# 'medium.bin' contains 1M of random 1's
print("Timing 'medium.bin'")
time = tt.timeit(count_medium_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()

### This runs fine in HW
# 'large.bin' contains 10M of random 1's
print("Timing 'large.bin'")
time = tt.timeit(count_large_hw, number=1)
print("Total Time:" + str(time) + " seconds")
print()